In [1]:
# exploring data attached 
import pandas as pd 


In [2]:
jobs = pd.read_excel("GA_task.xlsx")
jobs

,1,Unnamed: 1,2,Unnamed: 3,3,Unnamed: 5,4,Unnamed: 7,5,Unnamed: 9,...,46,Unnamed: 91,47,Unnamed: 93,48,Unnamed: 95,49,Unnamed: 97,50,Unnamed: 99
0,R,T,R,T,R,T,R,T,R,T,...,R,T,R,T,R,T,R,T,R,T
1,9,22,5,17,5,24,8,40,9,14,...,7,21,9,24,6,37,9,26,10,36
2,3,49,10,13,4,28,5,47,7,45,...,3,10,10,40,7,22,1,39,2,46
3,1,47,7,23,1,18,5,47,6,30,...,8,22,9,24,3,39,6,31,4,16
4,9,30,4,29,5,32,7,18,2,20,...,7,27,7,22,2,49,10,13,5,18
5,5,21,6,27,4,17,7,27,10,42,...,7,50,4,26,4,19,1,20,2,21
6,3,19,8,33,10,31,2,43,1,29,...,10,29,8,17,3,38,10,21,2,33
7,2,18,2,49,8,50,7,33,4,21,...,10,33,7,12,6,24,5,25,8,46
8,7,45,9,39,7,34,1,10,9,35,...,10,11,6,42,8,20,4,18,3,40
9,6,10,1,15,5,38,1,35,7,43,...,5,13,2,24,4,48,6,40,7,27


So to not get to confused with vocabulary. We face following problem we have 50 `tasks` each of them requires Resource R for some period of time T. Let's call one single element of this Task a Job for each Task Jobs need to be performed within declared order. Each resource we'll call a Machine so for job to be done we need some working time on specified machine.

In [3]:

class Task: 
    def __init__(self , jobs: list , id:int):
        """
        Represents a single task that requries some jobs within order to be done 
        Jobs a list of pair R - T for this task 
        """
        self.jobs = jobs                 # Jobs in order to be performed
        self.id = id            # as a Id in excel file is 1 -index and python is  0 - index
        self.current_timestamp = 0     # in our timeline kinda time mark at which we're with task so for example if we start first job at time 0 and 15 T is required we set this to 15.                        
    def __str__(self) -> str:
        return f"Task id: {self.id} \n jobs to perform: \n {self.jobs}"
    
    def return_machines(self):
        machines_required = []
        for job in self.jobs:
            machines_required.append(job[0])
        return machines_required
    
    def get_timestamp(self):
        return self.current_timestamp
    
    def get_machine_ix(self):
        return  self.jobs[0][0] - 1       # first job -> resource (1 index) - 1 to get 0_ix 
    
    def get_T(self):
        return  self.jobs[0][1]
    
    def next_step(self,time):              # we update our task after scheduling of first job to do
        self.jobs.pop(0)
        self.current_timestamp = time
    
   
        

In [4]:
class Machine:
    """Initialize machine object """
    def __init__(self, ix):
        self.id = ix   
        self.busy_until = 0   
        self.schedule =  []  # for now let's separete self.tasks for plot or even omit and create another parameter with let's call it occupation slots start-end of job performed on machine
        self.tasks = [[],[],[],[]]  # we'll try to perform visualization in a gant chart style, probably will work only on some small subset of our tasks 
        # format to then convert it to dataframe and utilize plotly so [[machine_id],[task_id],[start_time],[end_time]
        self.schedule.append((0,0)) # for consistency let's assume that on start machine is occupied with start at 0 and end at 0
        # self.task_scheduled = 1  # we now that at init task list has one trivial task
    def get_timestamp(self):
        return self.busy_until
        
    # let's also here add method to give a machine abilty to return slot at which it can perform given task, 
    def schedule_task(self,timestamp,time):
        """Timestamp first point in time at which task can be scheduled, time -> for task big T - time to perform job"""
        # we need to keep schedule in order 
        length = len(self.schedule)
        if length == 1:
            self.schedule.append((timestamp,timestamp+time))
            self.busy_until = timestamp+time
            self.tasks[2].append(timestamp)                        # getting our table for chart ready probably to improve
            self.tasks[3].append(timestamp+time)
            self.tasks[0].append(self.id)
            # self.task_scheduled +=1
            return timestamp+time
        else:
            # self.task_sch
            # eduled +=1
            for i in range(1,length):
                end_of_previous_job = self.schedule[i-1][1]
                start_of_next_job = self.schedule[i][0]
             
                if ((timestamp <= end_of_previous_job) & (end_of_previous_job+time <=start_of_next_job)):   # checking two conditions to answer question Can we fit new task somewhere between old scheduled tasks
                    self.schedule.insert(i,(end_of_previous_job,end_of_previous_job+time))   # put task into gap
                    
                    self.tasks[2].append(end_of_previous_job)
                    self.tasks[3].append(end_of_previous_job+time)
                    # only situation that end of current job in the task will differ from machine.busy_unitl
                    self.tasks[0].append(self.id)
                    # print(timestamp+time)
                    return end_of_previous_job+time # we need this to update task timeline 
                # polish cuz tired -> tu opisaliśmy sytuację gdzie możemy zacząć zadanie w trakcie kiedy jeszcze wyonujemy coś na maszynie, ale zmiescimy zadanie przed kolejnym już zaplanowanym 
                # mozemy jeszcze mieć sytuację, ze zaczniemy zadanie w trakcie przerwy, ale będizemy mogli rozpocząć dopiero po jakimś czasie od zwolnienia maszyny, ale i tak się wyrabiamy przed kolejnym zaplanowanym
                elif((end_of_previous_job <= timestamp <= start_of_next_job) &  (timestamp+time <=start_of_next_job)):
                    
                    self.schedule.insert(i,(timestamp,timestamp+time))   # put task into gap
                    self.tasks[2].append(timestamp)
                    self.tasks[3].append(timestamp+time)
                    # only situation that end of current job in the task will differ from machine.busy_unitl
                    self.tasks[0].append(self.id)
                    # print(timestamp+time)
                    return timestamp+time # we need this to update task timeline 
                
                
                    
            else:
                # we need to append at the end of task list for the machine but also first check if task.timepoint (end of previous job) > machine.timepoint(end of last scheduled task)
                if (timestamp<self.busy_until):
                    self.schedule.append((self.busy_until,self.busy_until+time))
                    
                    self.tasks[2].append(self.busy_until)
                    self.tasks[3].append(self.busy_until+time)
                    
                    self.busy_until += time
                    self.tasks[0].append(self.id)
                    return self.busy_until
                else:
                    self.schedule.append((timestamp,timestamp+time))
                    self.busy_until = timestamp + time
                    self.tasks[2].append(timestamp)
                    self.tasks[3].append(timestamp+time)
                    self.tasks[0].append(self.id)
                    return timestamp+time
        
        # print(self.busy_until)
        # return self.busy_until # in every other situation we can return self.busy_until

    def update_task_list(self,task_id):
        # during step of adding we also add task_id to the table:
        self.tasks[1].append(task_id+1) 
        
    def df_task_list(self):
        return pd.DataFrame({'machine_id':self.tasks[0],'task_id':self.tasks[1],'start_time':self.tasks[2],'end_time':self.tasks[3]})
    
    # def next_step(self):
    #     self.task_scheduled += 1
        
        
        

In [5]:
import random
import copy
from collections import Counter

def read_tasks(n = None):
    jobs = pd.read_excel("GA_task.xlsx")
    table = []
    col_number = len(jobs.columns)
  
    if n:
        length = 0
        for i in range(0, col_number,2):
            id = jobs.columns[i]
            task_jobs = list(zip(jobs.iloc[1:,i],jobs.iloc[1:,i+1]))
            table.append(Task(jobs=task_jobs,id = id)) # we add zipped pairs of machine and time for each task , and we convert to 0 index
            length +=1
            if length == n: return table
        
    for i in range(0, col_number,2):
        id = jobs.columns[i]
        task_jobs = list(zip(jobs.iloc[1:,i],jobs.iloc[1:,i+1]))
        table.append(Task(jobs=task_jobs,id = id)) # we add zipped pairs of machine and time for each task , and we convert to 0 index
    
    return table


def extract_machines(task_table: list):
    """ Requires task_table -> list of Task objects"""
    list_of_machines = []
    for task in task_table:
        machines_for_task = task.return_machines()   # this method returns list of machines for this task
        for machine in machines_for_task:
            list_of_machines.append(machine)   # we append machine requried
    # in the end remove duplicates and return 
    return list(set(list_of_machines))
    
    
def shuffle(input_list, count):
    '''Shuffles any n number of values in a list'''
    indices_to_shuffle = random.sample(range(len(input_list)), k=count)
    to_shuffle = [input_list[i] for i in indices_to_shuffle]
    random.shuffle(to_shuffle)
    for index, value in enumerate(to_shuffle):
        old_index = indices_to_shuffle[index]
        input_list[old_index] = value
    return input_list

# order crossover
def crossover(geno1, geno2,len_crossover):
    # we can set a len of crossover 
    #Order crossover (OX1)
    geno1 = copy.deepcopy(geno1)   
    geno2 = copy.deepcopy(geno2)  
    
    size = len(geno1.geno)
    
    ixs = random.sample(range(0,size),len_crossover)   # here we store indexes from first parent  we want to place this elements in the child
    child = [None] * size
    
    # now we supply child with elements from first parent 
    
    occurences = {}
    
    for ix in ixs:
        child[ix] = geno1.geno[ix]
        if child[ix] in occurences:
            occurences[child[ix]] +=1
        else:
            occurences[child[ix]] = 1
        
    

        # we'll also store elements that we already have in child to than remove it from parent 2 starting from the end, that will guarantee correct geno
        
    parent2 = geno2.geno
    parent2.reverse()
    
    
    for task in occurences:
        
        # now to removing 
        while occurences[task] != 0:
            parent2.remove(task)
            occurences[task] -= 1 

    parent2.reverse() # back to correct order
    # now just add to child none spots items from parent2 as we made sure that we have number of items that we need 
    
    for index in range(len(child)):
        if child[index] is None:
            child[index] = parent2.pop(0)

            
    # now we got genotype 
    
    geno1.geno = child
    
        
        
    
    
    # now we need to fill child with missing elements in order that occur in second parent
    
    
    
    return geno1 
    
    
    

    
    

    
    
    

In [6]:
task_table = read_tasks()
machines = extract_machines(task_table)
machine_table = []

# now we are ready to initialize table of machines 

for ix in machines:
    machine_table.append(Machine(ix = ix))

In [7]:
# and now finally Genotype here we'll probably make the most of calcualtion and also for each genotype we need to calcualte time to complete tasks in given order 
# so to do that we'll utilize Machine and Task classes and their methods
import random
import copy
class Genotype:
    def __init__(self,task_table,machine_table,n_jobs=11,seed=None):
        """ It needs to be initialized with tables of Tasks and Machines and number of jobs in task"""
        # with assumption that each task requires same number of jobs length of genotype will be len(task_table) * len(machine_table)
        if seed is not None:
            random.seed(seed)
        self.geno = self.initialize_geno(len(task_table),n_jobs)
        self.task_table = copy.deepcopy(task_table)
        self.machine_table = copy.deepcopy(machine_table)  # to avoid manuplating original input and then reinitializing it and so on 
        self.time = 0 
      
    def initialize_geno(self,n,k):
        geno = list(range(0,n)) * k     # just creating random genotype 
        random.shuffle(geno)
        return geno
        
    def calculate_time(self): 
        for task_ix in self.geno:
            machine_ix = self.task_table[task_ix].get_machine_ix()  # extracting first machine_x from task
            task_timestamp = self.task_table[task_ix].get_timestamp()   # extracting current position in time of this task end
            # machine_timestamp = self.machine_table[machine_ix].get_timestamp()  # extracting current position in time of machine required
            # # two options
            time = self.task_table[task_ix].get_T() # getting Time of job
            
            task_till = self.machine_table[machine_ix].schedule_task(task_timestamp,time)   # this function will return point in time at which we're with this task
            # just for future visualization
            self.machine_table[machine_ix].update_task_list(task_ix)
            # and now we need to pop this job from task requirments It's done or at least planned to be done
            self.task_table[task_ix].next_step(task_till)   # we remove first job from job list for this task and we update point in time of this task
            
            # print(f"Task_id: {task_ix}, Machine: {machine_ix} , zadanioe zaplanowane do: {task_till}, zadanie może się rozpocząć: {task_timestamp}, maszyna zajęta do {self.machine_table[machine_ix].busy_until}")
            # self.machine_table[machine_ix].next_step()   # just keep tracking of number scheduled jobs for machine
             
        # at the end of this loop we have all machines in table scheduled we now we can extract time at which each of them finish and max of it is time in this genotype
        
        max = 0
        for machine in self.machine_table:
            if (machine.busy_until > max):
                max = machine.busy_until
        self.time = max
        return self.time
    
    def mutate(self,percent = 10):
        """Function that we call to mutate genotype (random shuffle some part of it)

        Args:
            percent (int) percent of genotype we want to shuffle for example 20 means shuffle 1/5 elements of geno. In algorithm we'll have one more variable because 
            we''ll be able to decide how many genotype mutate
        """
        
        n_to_shuffle = len(self.geno) // percent
        
        self.geno = shuffle(self.geno,n_to_shuffle)
        
        
        
    
    
    
    def reset(self,task_table,machine_table):   # it doesnt reset geno
        # self.geno = self.initialize_geno(len(task_table),n_jobs)
        self.task_table = copy.deepcopy(task_table)
        self.machine_table = copy.deepcopy(machine_table)  # to avoid manuplating original input and then reinitializing it and so on 
        self.time = 0 
                
        
        
            
            
        
        
        
        
        


In [12]:
# genetic algorithm 
#population -> mutation -> crossover -> genotype to fenotype treansition that's our interprantation of schedule and then calcualte time as a fitness function
# -> probability of survival -> selection 
import copy

def genethic_algorithm(population_size ,  task_table , machine_table, survival_number = 4, steps_number = None ,mutation_percent= None , percent_to_cross = 50):
    
    # initialize population 
    
    population = [Genotype(task_table,machine_table,n_jobs=11) for _ in range(population_size)]
    
    gen_length = len(population[0].geno)
    
    # calculate fitness 
    
    fitness_table = [(population[ix],population[ix].calculate_time()) for ix in range(population_size)]#.sort(key=lambda x: x[1])
    
    #fitness_table = fitness_table.sort(key=lambda x: x[1])
    fitness_table.sort(key=lambda x: x[1])  # sorting by ascending time
    
    print(f"Step 0: best time = {fitness_table[0][1]} \n")

    # we'll always keep at least some part of survival without any changes but make sure to mutate at least some of surivals
    for _ in range(1,steps_number):
        
        new_population = [x[0] for x in fitness_table[:survival_number]]   # tu mamy 4 najlepsze 
        
        
        
        for item in new_population:
            item.reset(task_table,machine_table)
        
    
        # mutacje 
        
        for genotype in range(survival_number):    # mutatcja
            mutation = copy.deepcopy(new_population[genotype])
            mutation.reset(task_table,machine_table)
            mutation.mutate(percent=mutation_percent)
            new_population.append(mutation)
            
        # let's add crossovers 
        

        
        
        for i in range(1,survival_number,2):
            
            cross = crossover(new_population[i-1],new_population[i],len_crossover = percent_to_cross * gen_length // 100)
            # cross1.reset(task_table=task_table,machine_table=machine_table)
            cross2 = crossover(new_population[i],new_population[i-1],len_crossover = percent_to_cross * gen_length // 100)
            # cross2.reset(task_table=task_table,machine_table=machine_table)
            new_population.append(cross)
            new_population.append(cross2)            
       
        
            
        # and append 2 more random elements, from testing it seems to improve performance 
        
        new_population.append(Genotype(task_table=task_table,machine_table=machine_table,n_jobs=11))
        new_population.append(Genotype(task_table=task_table,machine_table=machine_table,n_jobs=11))
        
        # return new_population ,2
        size = len(new_population)
        # return new_population[8],2
        fitness_table = [(new_population[ix],new_population[ix].calculate_time()) for ix in range(size)]#.sort(key=lambda x: x[1])
        # fitness_table = fitness_table.sort(key=lambda x: x[1])
        fitness_table.sort(key=lambda x: x[1])  # sorting by ascending tim
        
        
        
        print(f"Step {_}: best time = {fitness_table[0][1]} \n")
            
        # errors 
    
    # at the end let's return top 5 of fitness table
    
    return fitness_table[:5]  
    
    

In [ ]:
top_5_table  = genethic_algorithm(15,task_table=task_table,machine_table=machine_table,mutation_percent=10,percent_to_cross=50,steps_number=20)

Step 0: best time = 2183 

Step 1: best time = 2146 

Step 2: best time = 2124 

Step 3: best time = 2124 

Step 4: best time = 2119 

Step 5: best time = 2119 

Step 6: best time = 2114 

Step 7: best time = 2114 

Step 8: best time = 2093 

Step 9: best time = 2093 

Step 10: best time = 2093 

Step 11: best time = 2093 

Step 12: best time = 2093 

Step 13: best time = 2093 

Step 14: best time = 2093 

Step 15: best time = 2093 

Step 16: best time = 2093 

Step 17: best time = 2093 

Step 18: best time = 2093 

Step 19: best time = 2093 

Step 20: best time = 2093 

Step 21: best time = 2093 

Step 22: best time = 2093 

Step 23: best time = 2093 

Step 24: best time = 2093 

Step 25: best time = 2093 

Step 26: best time = 2093 

Step 27: best time = 2093 

Step 28: best time = 2093 

Step 29: best time = 2093 

Step 30: best time = 2093 

Step 31: best time = 2093 

Step 32: best time = 2093 

Step 33: best time = 2093 

Step 34: best time = 2093 

Step 35: best time = 2093 

St

In [14]:
steps_1000 = genethic_algorithm(15,task_table=task_table,machine_table=machine_table,mutation_percent=20,percent_to_cross=25,steps_number=1000)

Step 0: best time = 2131 

Step 1: best time = 2131 

Step 2: best time = 2131 

Step 3: best time = 2131 

Step 4: best time = 2131 

Step 5: best time = 2131 

Step 6: best time = 2103 

Step 7: best time = 2103 

Step 8: best time = 2103 

Step 9: best time = 2103 

Step 10: best time = 2103 

Step 11: best time = 2103 

Step 12: best time = 2103 

Step 13: best time = 2099 

Step 14: best time = 2099 

Step 15: best time = 2099 

Step 16: best time = 2099 

Step 17: best time = 2099 

Step 18: best time = 2099 

Step 19: best time = 2095 

Step 20: best time = 2095 

Step 21: best time = 2095 

Step 22: best time = 2095 

Step 23: best time = 2095 

Step 24: best time = 2095 

Step 25: best time = 2095 

Step 26: best time = 2095 

Step 27: best time = 2095 

Step 28: best time = 2095 

Step 29: best time = 2095 

Step 30: best time = 2095 

Step 31: best time = 2095 

Step 32: best time = 2095 

Step 33: best time = 2095 

Step 34: best time = 2095 

Step 35: best time = 2095 

St

In [11]:
testowe = genethic_algorithm(30,task_table=task_table,machine_table=machine_table,mutation_percent=30,percent_to_cross=25,steps_number=200,survival_number=8)

Step 0: best time = 2165 

Step 1: best time = 2152 

Step 2: best time = 2108 

Step 3: best time = 2108 

Step 4: best time = 2108 

Step 5: best time = 2108 

Step 6: best time = 2108 

Step 7: best time = 2108 

Step 8: best time = 2108 

Step 9: best time = 2108 

Step 10: best time = 2108 

Step 11: best time = 2108 

Step 12: best time = 2108 

Step 13: best time = 2108 

Step 14: best time = 2108 

Step 15: best time = 2108 

Step 16: best time = 2108 

Step 17: best time = 2108 

Step 18: best time = 2108 

Step 19: best time = 2108 

Step 20: best time = 2107 

Step 21: best time = 2107 

Step 22: best time = 2107 

Step 23: best time = 2107 

Step 24: best time = 2107 

Step 25: best time = 2107 

Step 26: best time = 2107 

Step 27: best time = 2107 

Step 28: best time = 2107 

Step 29: best time = 2107 

Step 30: best time = 2107 

Step 31: best time = 2107 

Step 32: best time = 2107 

Step 33: best time = 2107 

Step 34: best time = 2107 

Step 35: best time = 2107 

St

In [26]:
genethic_algorithm(15,task_table=task_table,machine_table=machine_table,mutation_percent=20,percent_to_cross=25,steps_number=20)

Step 0: best time = 2145 

Step 1: best time = 2145 

Step 2: best time = 2145 

Step 3: best time = 2145 

Step 4: best time = 2142 

Step 5: best time = 2137 

Step 6: best time = 2137 

Step 7: best time = 2110 

Step 8: best time = 2110 

Step 9: best time = 2110 

Step 10: best time = 2110 

Step 11: best time = 2110 

Step 12: best time = 2110 

Step 13: best time = 2110 

Step 14: best time = 2110 

Step 15: best time = 2110 

Step 16: best time = 2110 

Step 17: best time = 2110 

Step 18: best time = 2110 

Step 19: best time = 2110 



[(<__main__.Genotype at 0x197f217f470>, 2110),
 (<__main__.Genotype at 0x197f36a6360>, 2118),
 (<__main__.Genotype at 0x197f36ad7f0>, 2129),
 (<__main__.Genotype at 0x197f2684530>, 2131),
 (<__main__.Genotype at 0x197f36ab2c0>, 2134)]

In [27]:
genethic_algorithm(15,task_table=task_table,machine_table=machine_table,mutation_percent=50,percent_to_cross=50,steps_number=20)

Step 0: best time = 2160 

Step 1: best time = 2160 

Step 2: best time = 2160 

Step 3: best time = 2145 

Step 4: best time = 2145 

Step 5: best time = 2145 

Step 6: best time = 2145 

Step 7: best time = 2145 

Step 8: best time = 2143 

Step 9: best time = 2143 

Step 10: best time = 2143 

Step 11: best time = 2143 

Step 12: best time = 2143 

Step 13: best time = 2143 

Step 14: best time = 2143 

Step 15: best time = 2143 

Step 16: best time = 2137 

Step 17: best time = 2128 

Step 18: best time = 2128 

Step 19: best time = 2128 



[(<__main__.Genotype at 0x197f36abec0>, 2128),
 (<__main__.Genotype at 0x197f2681220>, 2132),
 (<__main__.Genotype at 0x197f36a50a0>, 2133),
 (<__main__.Genotype at 0x197f219dc10>, 2133),
 (<__main__.Genotype at 0x197f36a9ac0>, 2137)]

In [16]:
top_5_table

[(<__main__.Genotype at 0x197f16b8320>, 2104),
 (<__main__.Genotype at 0x197f0642db0>, 2115),
 (<__main__.Genotype at 0x197f05457c0>, 2116),
 (<__main__.Genotype at 0x197f05da540>, 2123),
 (<__main__.Genotype at 0x197f063f770>, 2124)]

In [22]:
best = top_5_table[0][0]

In [24]:
from plotly.express import bar
def visualize_schedule(machine_table,total_time):
    """ Requires machine table (schedule) for specific genotype"""
    df_list = []
    for i in machine_table:
        df_list.append(i.df_task_list())
    df_sumup = pd.concat(df_list,axis=0)
    df_sumup['end'] = df_sumup['end_time'] - df_sumup['start_time']
    fig = bar(data_frame=df_sumup,base='start_time',x = 'end',y = 'machine_id',color = "task_id",orientation='h',title=f"Total time: {total_time}")
    fig.show()
visualize_schedule(best.machine_table,best.time)

In [31]:
steps_1000_best = steps_1000[0][0]
visualize_schedule(steps_1000_best.machine_table,steps_1000_best.time)

In [37]:
testowe_best = testowe[0][0]
visualize_schedule(testowe_best.machine_table,testowe_best.time)

In [19]:
random_geno = Genotype(task_table=task_table,machine_table=machine_table)
random_geno.calculate_time()

2274

In [20]:
visualize_schedule(random_geno.machine_table,random_geno.time)